# 3. przygotowanie danych do transfer learningu

In [ ]:
from scipy.io import loadmat
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split
import random
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import statistics
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import pickle
import torchvision.transforms as T
import scipy.misc

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#sciagniecie danych z google drive
!unzip "/content/gdrive/MyDrive/Colab Notebooks/car_ims.zip" -d "/content"

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: /content/car_ims/004288.jpg  
  inflating: /content/__MACOSX/car_ims/._004288.jpg  
  inflating: /content/car_ims/002821.jpg  
  inflating: /content/__MACOSX/car_ims/._002821.jpg  
  inflating: /content/car_ims/015385.jpg  
  inflating: /content/__MACOSX/car_ims/._015385.jpg  
  inflating: /content/car_ims/005196.jpg  
  inflating: /content/__MACOSX/car_ims/._005196.jpg  
  inflating: /content/car_ims/002612.jpg  
  inflating: /content/__MACOSX/car_ims/._002612.jpg  
  inflating: /content/car_ims/012401.jpg  
  inflating: /content/__MACOSX/car_ims/._012401.jpg  
  inflating: /content/car_ims/004263.jpg  
  inflating: /content/__MACOSX/car_ims/._004263.jpg  
  inflating: /content/car_ims/014070.jpg  
  inflating: /content/__MACOSX/car_ims/._014070.jpg  
  inflating: /content/car_ims/000005.jpg  
  inflating: /content/__MACOSX/car_ims/._000005.jpg  
  inflating: /content/car_ims/010216.jpg  
  inflating: /conte

In [ ]:
#budowa funkcji, ktorej celem jest prawidlowe sciagniecie pliku zawierajacego 
#etykiety zbioru wykorzystanego do analizy 
def read_mat_to_df(file_path):
    np_array= loadmat(file_path, matlab_compatible=False, simplify_cells=True, chars_as_strings=True)
    df= pd.DataFrame(np_array['annotations'])
    if 'class_names' in np_array:
        class_names= list(np_array['class_names'])
        df['class_name']= df['class'].map(dict(enumerate(class_names, start=1)))
    # squeeze int-types
    for c, t in df.dtypes.iteritems():
        if t.kind == 'i':
            df[c]= df[c].astype('int16')
    if 'fname' in df.columns:
        df['relative_im_path']= 'car_ims/0' + df['fname']
        df.drop(columns=['fname'], inplace=True)
    return df

cars_full= read_mat_to_df('gdrive/MyDrive/cars_annos.mat')
class_class_name_count=cars_full[['class','class_name']].value_counts().reset_index()
class_to_name_dict=dict(zip(class_class_name_count['class'], class_class_name_count.class_name))

<ipython-input-6-0539c57b4ae5>:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for c, t in df.dtypes.iteritems():


In [ ]:
#utworzenie kolumny w tabeli z etykietami, ktora bedzie wykorzystana do analizy jako zmienna objasniana
cars_full['brand']=[class_name.split()[0] for class_name in cars_full['class_name'].tolist()]
map_brand=cars_full['brand'].value_counts().reset_index()
map_brand['class']=[x for x in range(1,len(map_brand)+1)]
class_to_brand_dict=dict(zip(map_brand['index'], map_brand['class']))
cars_full['class']=cars_full['brand'].map(class_to_brand_dict)

In [ ]:
# zbudowanie funkcji, ktorej celem bedzie ekstrakcja danych. Funkcja ta posiada wiele argumentow, ktore
# umozliwiaja uzytkownikowi modyfikacje obserwacji
# cropped - przycinanie zdjecia aby wynikiem byl wylacznie samochod
# white_black - wybranie wymiarow zdjecia, czarne==True, biale==False
# sub_sample - wybranie ilosci zdjec poddanych ekstracji i pre-processingu
# image_dim_change - bool parametr, ktory uzytkownik moze uzyc w sytuacji checi zmienienia wymiarow zdjecia
# image_dim_change_val - parametr, ktory oznajmia jakie wymiary zdjecie powinno posiadac
# final_data - zastosowanie wyrownania histogramy oraz podzielenia wektoru ze zdjeciem przez 255 w sytuacji, gdy 
# parametr bedzie rownal sie z True
# wynikiem funkcji sa zdjecia oraz ich wymiary

def loop_for_car_images(df,white_black=True,sub_sample=None,
                        cropped=True, image_stand=False,
                        image_stand_val=[100,100],final_data=False):
    df1=df.copy()
    images=[]
    i=0
    images_dim=[]
    for index,row in df1.iterrows():
        i+=1
        path=os.path.join(df['relative_im_path'][index])
        image = Image.open(path)
        if len(np.array(image).shape)==3 and white_black==True:
            image=image.convert('L')
        image_array=np.array(image)
        if len(image_array.shape)<3 and white_black==False:
            image_array=cv2.merge((image_array,image_array,image_array))
        if cropped:
            image_array=image_array[row['bbox_y1']:row['bbox_y2'],row['bbox_x1']:row['bbox_x2']]
        if image_stand:
            image_array = Image.fromarray(image_array)
            transform = T.Resize(size = (image_stand_val[0],image_stand_val[1]))
            image_array = transform(image_array)
            image_array=np.array(image_array)
        if final_data:
            image_array=image_array/255

        images.append(image_array)
        images_dim.append(image_array.shape)
        if i%1000==0:
            print(f'{i} photos have been uploaded')
    return images, images_dim

In [ ]:
#sciaganie danych oraz ich wymiarow
images,_=loop_for_car_images(cars_full,white_black=False,image_stand=True,
                              image_stand_val=[200,200],final_data=True)


1000 photos have been uploaded
2000 photos have been uploaded
3000 photos have been uploaded
4000 photos have been uploaded
5000 photos have been uploaded
6000 photos have been uploaded
7000 photos have been uploaded
8000 photos have been uploaded
9000 photos have been uploaded
10000 photos have been uploaded
11000 photos have been uploaded
12000 photos have been uploaded
13000 photos have been uploaded
14000 photos have been uploaded
15000 photos have been uploaded
16000 photos have been uploaded


In [ ]:
# podzial danych na zbior treningowy, walidacyjny oraz testowy
X=images
y=cars_full['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=5,stratify=y)
X_val, X_test, y_val, y_test =train_test_split(X_test, y_test, test_size = 0.5,random_state=20,stratify=y_test)
del images, _, X, y

In [ ]:
# utworzenie funkcji, ktora stosowac bedzie na danych metody data augmentation
augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2,fill_mode='constant',seed=1),
])
def data_aug(X,y,how_much_total,show=False):
    data_val_count=y.value_counts().reset_index()
    low_cat=data_val_count[data_val_count['class']<how_much_total]
    low_cat.columns=['class','count']
    images_aug=list(X.copy())
    labels_aug=list(y.copy())
    df_y=pd.DataFrame(y).reset_index()
    df_y.columns=['previous index','y']
    for _,low_cat_row in low_cat.iterrows():
        how_much_aug=how_much_total-low_cat_row['count']
        df_temp=df_y[df_y['y']==low_cat_row['class']]
        i=0
        show1=False
        if show==True:
            show1=True
        while i<how_much_aug:
            for index,row in df_temp.iterrows():
                #image_to_aug=X[index].copy()[..., np.newaxis]
                image_to_aug=X[index].copy()
                aug_image=augmentation(image_to_aug)
                if show1:
                    print(class_to_name_dict.get(low_cat_row['class']))
                    plt.figure(figsize=(15,10))
                    plt.imshow(image_to_aug)
                    plt.show()
                    plt.figure(figsize=(15,10))
                    plt.imshow(aug_image)
                    plt.show()
                    show1=False
                images_aug.append(aug_image)
                labels_aug.append(low_cat_row['class'])
                i+=1
                if i>=how_much_aug:
                    break
    print('Done!')
    return images_aug, labels_aug

In [ ]:
# zastosowanie data augmentation na danych treningowych
X_train, y_train=data_aug(X_train,y_train,how_much_total=250,show=False)


Done!


In [ ]:
# zmienienie klasy danych aby mogly byc uzyte do budowy modeli splotowych sieci neuronowych

X_train=np.array(X_train)
X_val=np.array(X_val)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_val=np.array(y_val)
y_test=np.array(y_test)


y_train=tf.keras.utils.to_categorical(y_train)
y_val=tf.keras.utils.to_categorical(y_val)
y_test=tf.keras.utils.to_categorical(y_test)

In [ ]:
# zapisanie dancyh w formie pickle

import pickle
pick_insert = open('gdrive/My Drive/X_train_color.pkl','wb')
pickle.dump(X_train, pick_insert)
pick_insert.close()
pick_insert = open('gdrive/My Drive/X_val_color.pkl','wb')
pickle.dump(X_val, pick_insert)
pick_insert.close()
pick_insert = open('gdrive/My Drive/X_test_color.pkl','wb')
pickle.dump(X_test, pick_insert)
pick_insert.close()
pick_insert = open('gdrive/My Drive/y_train_color.pkl','wb')
pickle.dump(y_train, pick_insert)
pick_insert.close()
pick_insert = open('gdrive/My Drive/y_val_color.pkl','wb')
pickle.dump(y_val, pick_insert)
pick_insert.close()
pick_insert = open('gdrive/My Drive/y_test_color.pkl','wb')
pickle.dump(y_test, pick_insert)
pick_insert.close()